<a href="https://colab.research.google.com/github/mudesir/DS-Unit-1-Sprint-2-Statistics/blob/master/Mudesir_LS_DS17_122_Chi2_Tests_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment - Practice Chi^2 Tests

Use the following dataset relating to math scores of students in two different Portugese schools:

<https://archive.ics.uci.edu/ml/datasets/Student+Performance>

### 1) Load the dataset specifically relating to math scores as a new dataframe.

There are two datasets in the `student.zip` file, make sure you use `student-mat.csv`.


In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip

--2020-06-10 18:39:48--  https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20478 (20K) [application/x-httpd-php]
Saving to: ‘student.zip’

student.zip         100%[===================>]  20.00K  --.-KB/s    in 0.06s   

2020-06-10 18:39:48 (312 KB/s) - ‘student.zip’ saved [20478/20478]



In [3]:
! unzip student.zip

Archive:  student.zip
  inflating: student-mat.csv         
  inflating: student-por.csv         
  inflating: student-merge.R         
  inflating: student.txt             


In [14]:
# YOUR WORK HERE
import pandas as pd
import numpy as np
from scipy.stats import chisquare
from scipy import stats
df = pd.read_csv('student-mat.csv', sep=';')
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10


### 2) Use Chi^2 tests and `stats.chi2_contingency()` to identify:
 - Two pairs of variables that are dependent (are associated with one another).
 - Two pairs of variables that are independent (have no significant relationship).

Does it make intuitive sense why the variables in these pairs might or might not show a relationship to one another? 


In [16]:
df['schoolsup'].value_counts()

no     344
yes     51
Name: schoolsup, dtype: int64

In [17]:
df['famsup'].value_counts()

yes    242
no     153
Name: famsup, dtype: int64

In [19]:

contingency = pd.crosstab(df['schoolsup'], df['famsup'])

contingency

famsup,no,yes
schoolsup,,
no,140,204
yes,13,38


In [20]:
chi2, p_value, dof, expected = stats.chi2_contingency(contingency)

print("chi2 statistic", chi2)
print("p value", p_value)
print("degrees of freedom",dof)
print("expected frequencies table", expected)

chi2 statistic 3.711345590895456
p value 0.05404380626633763
degrees of freedom 1
expected frequencies table [[133.24556962 210.75443038]
 [ 19.75443038  31.24556962]]


Conclusion: There is no evidence to reject that  schoolsup - extra educational support and famsup - family educational support  are associated (show a relationship). They might be independent.

### 3) Use NumPy to perform your own chi^2 test "from scratch" 

Pick any of the chi^2 tests that you ran in part 2 and try them on your own. You should get the same results that Scipy got for all four values returned from `chi2_contingency()`

In [24]:
# YOUR WORK HERE
df['Walc'].value_counts()

1    151
2     85
3     80
4     51
5     28
Name: Walc, dtype: int64

In [31]:
df['health'].value_counts().sort_index()

1     47
2     45
3     91
4     66
5    146
Name: health, dtype: int64

In [32]:
observed = pd.crosstab(df['Walc'], df['health'])

observed

health,1,2,3,4,5
Walc,,,,,
1,20,15,41,28,47
2,11,8,20,13,33
3,9,14,18,12,27
4,5,6,7,5,28
5,2,2,5,8,11


## Expected Value Calculation
\begin{align}
expected_{i,j} =\frac{(row_{i} \text{total})(column_{j} \text{total}) }{(\text{total observations})}  
\end{align}

In [33]:
observed_with_margins = pd.crosstab(df['Walc'], df['health'], margins=True)

observed_with_margins

health,1,2,3,4,5,All
Walc,,,,,,
1,20,15,41,28,47,151
2,11,8,20,13,33,85
3,9,14,18,12,27,80
4,5,6,7,5,28,51
5,2,2,5,8,11,28
All,47,45,91,66,146,395


In [42]:
row_totals = observed_with_margins.iloc[5:6, 0:5].values[0]

row_totals

array([ 47,  45,  91,  66, 146])

In [48]:
col_totals = df['Walc'].value_counts().sort_index().values

col_totals

array([151,  85,  80,  51,  28])

In [59]:
sample_size = observed_with_margins.iloc[5:6, 5:6].values[0]
sample_size

array([395])

In [61]:
expected = np.array([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],
                     [0,0,0,0,0],[0,0,0,0,0]])

for i, col in enumerate(col_totals):
  for j, row in enumerate(row_totals): 
    expected_value = (row*col / sample_size)
    # print(expected_value, i, j)
    expected[i][j] = expected_value

expected

array([[17, 17, 34, 25, 55],
       [10,  9, 19, 14, 31],
       [ 9,  9, 18, 13, 29],
       [ 6,  5, 11,  8, 18],
       [ 3,  3,  6,  4, 10]])

In [34]:
observed = observed.values
observed

array([[20, 15, 41, 28, 47],
       [11,  8, 20, 13, 33],
       [ 9, 14, 18, 12, 27],
       [ 5,  6,  7,  5, 28],
       [ 2,  2,  5,  8, 11]])

### Calculate the chi^2 statistic (test statistic)

\begin{align}
\chi^2 = \sum \frac{(observed_i-expected_i)^2}{(expected_i)}
\end{align}

In [62]:
((observed - expected)**2 / expected).sum().sum()

20.621455135413733

In [0]:
# dof = (5-1)*(5-1)=16
# p-value with 16 dof from table
pvalue=0.192

Conclusion: Based on a chi2 statistic of 20.62, and a p-value of .192, we fail to reject the null hypothesis that Walc(weekly alchole consumption) and health(current health status) are independent.

## Stretch goals:

### 1. Refactor your code so it is elegant, readable, and holds reusable code in functions.

In [0]:
# YOUR WORK HERE



### 2. Check For Understanding - Study and write your own explanations/definitions for these topics:

- What is a sample "estimate" in statistics?

- What are hypothesis test? How are they useful?

- What is a "null hypothesis?"

- What is a p-value and what does it represent?

- What does it mean for something to be "statistically significant?"

- What is a test statistic and how does it relate to a p-value?

- What are "degrees of freedom" and how are they calculated in a 1-sample, 2-sample, and chi^2 test for independence?

## Resources

- [Interactive visualize the Chi-Squared test](https://homepage.divms.uiowa.edu/~mbognar/applets/chisq.html)
- [Calculation of Chi-Squared test statistic](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test)
- [Visualization of a confidence interval generated by R code](https://commons.wikimedia.org/wiki/File:Confidence-interval.svg)
- [Expected value of a squared standard normal](https://math.stackexchange.com/questions/264061/expected-value-calculation-for-squared-normal-distribution) (it's 1 - which is why the expected value of a Chi-Squared with $n$ degrees of freedom is $n$, as it's the sum of $n$ squared standard normals)